In [38]:
import os,json,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath("..")) 
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import re
from tqdm import tqdm
from utils.utils import *
from datasets import load_dataset
import ast
from collections import defaultdict
sys.path.append(os.path.abspath("aside")) # to access aside.experiments
from utils.model_utils import load_model
from vllm import SamplingParams

In [27]:
model,tokenizer,is_aside = load_model('Qwen/Qwen3-8B',use_vllm=True,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':False})

INFO 10-06 11:38:04 [utils.py:328] non-default args: {'max_model_len': 32768, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enable_chunked_prefill': False, 'model': 'Qwen/Qwen3-8B'}
INFO 10-06 11:38:23 [__init__.py:742] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-06 11:38:23 [__init__.py:1815] Using max model len 32768
INFO 10-06 11:38:27 [scheduler.py:222] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 10-06 11:38:30 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-06 11:38:42 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=89642) INFO 10-06 11:38:44 [core.py:654] Waiting for init message from front-end.
(EngineCore_DP0 pid=89642) INFO 10-06 11:38:44 [core.py:76] Initializing a V1 LLM engine (v0.10.2) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=a

[W1006 11:38:49.130278224 ProcessGroupNCCL.cpp:981] Warning: TORCH_NCCL_AVOID_RECORD_STREAMS is the default now, this environment variable is thus deprecated. (function operator())


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=89642) INFO 10-06 11:38:49 [parallel_state.py:1165] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
(EngineCore_DP0 pid=89642) WARNING 10-06 11:38:49 [topk_topp_sampler.py:69] FlashInfer is not available. Falling back to the PyTorch-native implementation of top-p & top-k sampling. For the best performance, please install FlashInfer.
(EngineCore_DP0 pid=89642) INFO 10-06 11:38:49 [gpu_model_runner.p

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:00<00:02,  1.91it/s]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:01<00:03,  1.08s/it]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [00:03<00:02,  1.50s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [00:06<00:01,  1.71s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:08<00:00,  1.83s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [00:08<00:00,  1.62s/it]
(EngineCore_DP0 pid=89642) 


(EngineCore_DP0 pid=89642) INFO 10-06 11:39:01 [default_loader.py:268] Loading weights took 8.29 seconds
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:02 [gpu_model_runner.py:2392] Model loading took 15.2683 GiB and 11.947722 seconds
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:12 [backends.py:539] Using cache directory: /export/home2/weijie210/.cache/vllm/torch_compile_cache/77df5711bc/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:12 [backends.py:550] Dynamo bytecode transform time: 9.59 s
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:26 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 12.834 s
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:27 [monitor.py:34] torch.compile takes 9.59 s in total
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:31 [gpu_worker.py:298] Available KV cache memory: 46.66 GiB
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:31 [kv_cache_utils.py:864] GPU KV cache size: 339,792 tokens
(EngineCo

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|█| 67/67 [00:08<0


(EngineCore_DP0 pid=89642) INFO 10-06 11:39:41 [gpu_model_runner.py:3118] Graph capturing finished in 10 secs, took 0.62 GiB
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:41 [gpu_worker.py:391] Free memory on device (78.76/79.25 GiB) on startup. Desired GPU memory utilization is (0.8, 63.4 GiB). Actual usage is 15.27 GiB for weight, 1.45 GiB for peak activation, 0.02 GiB for non-torch memory, and 0.62 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=49284854784` to fit into requested memory, or `--kv-cache-memory=65780986880` to fully utilize gpu memory. Current kv cache memory in use is 50104841216 bytes.
(EngineCore_DP0 pid=89642) INFO 10-06 11:39:41 [core.py:218] init engine (profile, create kv cache, warmup model) took 39.16 seconds
INFO 10-06 11:39:44 [llm.py:295] Supported_tasks: ['generate']
INFO 10-06 11:39:44 [__init__.py:36] No IOProcessor plugins requested by the model


In [4]:
ds = load_dataset('Agent-Ark/Toucan-1.5M','SFT',split = 'train').to_list()

Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

In [42]:
## reformat the dataset
toolcall_format = "<tool_call>\n{tool}\n</tool_call>\n"

cat_ds = defaultdict(list)
invalid_cats = defaultdict(int)
for sample in tqdm(ds,total = len(ds)):
    try: # have to be able to parse both of them.
        tools = json.loads(sample['tools'])
        messages = json.loads(sample['messages'])
    except:
        invalid_cats['parsing_error'] += 1
        continue
    
    cat = sample['subset_name']
    roles = [m['role'] for m in messages]
    if not any([r == 'tool_response' for r in roles]): # no tool roles
        invalid_cats['no_tool_use'] += 1
        continue
    
    structured_msg = []
    curr_tool_call = ''
    assistant_start = False
    for msg_id,m in enumerate(messages): # we ignore the assistant role since Qwen do not generate any text other than the MCP tools
        if m['role'] == 'tool_call':
            curr_tool_call += toolcall_format.format(tool = m['content'])
        else:
            if curr_tool_call != '': # if the next msg is not tool_call, we dump the curr_tool_call
                structured_msg.append({'role':'assistant','content':curr_tool_call.strip()})
                curr_tool_call = ''
            if m['role'] == 'user':
                structured_msg.append(m)
            elif m['role'] == 'tool_response':
                structured_msg.append({'role':'tool','content':m['content']})
            elif m['role'] == 'assistant': # if is assistant, we only keep it if it is the last msg or if the next msg is not tool_call.
                if msg_id == len(messages) - 1 or messages[msg_id + 1]['role'] != 'tool_call':
                    structured_msg.append(m)
                
            
    cat_ds[cat].append(
        {'conversations':structured_msg,
         'tools':tools,}
    )

for k in invalid_cats:
    pprint(f"Invalid cat {k}: {invalid_cats[k]} samples")

for k in cat_ds:
    pprint(f"Valid cat {k}: {len(cat_ds[k])} samples")
        
    

100%|██████████████████████████████████████████████████████████████████| 119287/119287 [00:18<00:00, 6367.68it/s]


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Invalid cat no_tool_use: 40000 samples                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Valid cat single-turn-original: 28254 samples                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Valid cat single-turn-diversify: 15790 samples                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Valid cat multi-turn: 35243 samples                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

In [46]:
example_msg = cat_ds['multi-turn'][10]['conversations']
pprint (tool_prompt_format(example_msg,cat_ds['multi-turn'][10]['tools'],tokenizer))
# for msg in example_msg:
#     pprint (f'Role: {msg["role"]}, Content: {msg["content"]}')

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ <|im_start|>system                                                                               │
│ # Tools                                                                                          │
│                                                                                                  │
│ You may call one or more functions to assist with the user query.                                │
│                                                                                                  │
│ You are provided with function signatures within <tools></tools> XML tags:                       │
│ <tools>                                                                                          │
│ {"type": "function", "function": {"name": "pokémcp-random-pokemon", "description": "Get a random │
│ Pokémon", "parameters": {"type": "object", "properties": {}, "additionalProperties": false,      │
│ "$schema": "http://json-schema.org/draft-07/schema#"}}}                                          │
│ {"type": "function", "function": {"name": "pokémcp-random-pokemon-from-region", "description":   │
│ "Get a random Pokémon from a specific region", "parameters": {"type": "object", "properties":    │
│ {"region": {"type": "string", "description": "The Pokémon region (e.g., kanto, johto, hoenn,     │
│ etc.)"}}, "required": ["region"], "additionalProperties": false, "$schema":                      │
│ "http://json-schema.org/draft-07/schema#"}}}                                                     │
│ {"type": "function", "function": {"name": "pokémcp-random-pokemon-by-type", "description": "Get  │
│ a random Pokémon of a specific type", "parameters": {"type": "object", "properties": {"type":    │
│ {"type": "string", "description": "The Pokémon type (e.g., fire, water, grass, etc.)"}},         │
│ "required": ["type"], "additionalProperties": false, "$schema":                                  │
│ "http://json-schema.org/draft-07/schema#"}}}                                                     │
│ {"type": "function", "function": {"name": "pokémcp-pokemon-query", "description": "Answer        │
│ natural language Pokémon queries", "parameters": {"type": "object", "properties": {"query":      │
│ {"type": "string", "description": "A natural language query about Pokémon"}}, "required":        │
│ ["query"], "additionalProperties": false, "$schema":                                             │
│ "http://json-schema.org/draft-07/schema#"}}}                                                     │
│ {"type": "function", "function": {"name": "bible-verse-access-server-get_random_bible_verse",    │
│ "description": "\n    Get a random Bible verse.\n\n    Args:\n        book_ids: Optional         │
│ comma-separated list of book IDs (e.g., \"GEN,JHN\")\n                 or special strings \"OT\" │
│ (Old Testament) or \"NT\" (New Testament)\n\n    Returns:\n        Random verse data including   │
│ reference, text, and translation info\n    ", "parameters": {"properties": {"book_ids":          │
│ {"default": null, "title": "Book Ids", "type": "string"}}, "title":                              │
│ "get_random_bible_verseArguments", "type": "object"}}}                                           │
│ {"type": "function", "function": {"name": "bible-verse-access-server-get_bible_verse",           │
│ "description": "\n    Get a specific Bible verse or passage.\n\n    Args:\n        reference:    │
│ Bible reference (e.g., \"John 3:16\", \"Genesis 1:1-3\")\n        translation: Translation       │
│ identifier (default: \"web\" for World English Bible)\n\n    Returns:\n        Verse data        │
│ including reference, text, and translation info\n    ", "parameters": {"properties":             │
│ {"reference": {"title": "Reference", "type": "string"}, "translation": {"default": "web",        │
│ "title": "Translation", "type": "string"}}, "required": ["r

In [44]:
# Get the length.
def get_len(sample):
    convs = sample['conversations']
    formatted = tool_prompt_format(convs,sample['tools'],tokenizer)
    sample['len'] =len(tokenizer.encode(formatted))
    return sample

# Use multi process to get the lengths
cat_ds_w_len = {k:async_process(get_len,cat_ds[k],workers=32, msg="Computing input lengths") for k in cat_ds}


Computing input lengths: 100%|████████████████████████████████████████████| 35243/35243 [01:03<00:00, 556.95it/s]


In [45]:
MAX_LENGTH = 4096
cat_ds_valid = {k:[s for s in cat_ds_w_len[k] if s['len'] <= MAX_LENGTH] for k in cat_ds_w_len}
for k in cat_ds_w_len:
    pprint (f"Cat {k}: before filtering {len(cat_ds_w_len[k])}, after filtering {len(cat_ds_valid[k])}")

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat single-turn-original: before filtering 28254, after filtering 22863                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat single-turn-diversify: before filtering 15790, after filtering 13314                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Cat multi-turn: before filtering 35243, after filtering 21562                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯